In [2]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema

# 1. Setup Data
symbol = "BTC-USD"
df = yf.download(symbol, period="60d", interval="1h")
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

# 2. Calculate RSI
df['RSI'] = ta.rsi(df['Close'], length=14)
df.dropna(inplace=True)


def detect_signals(df, order=5):
    """
    Detects local extrema and identifies divergence/convergence.
    order: how many candles on each side to define a peak/trough.
    """
    # Find local peaks and troughs in Price and RSI
    df['Price_Max'] = df.iloc[argrelextrema(
        df['Close'].values, np.greater, order=order)[0]]['Close']
    df['Price_Min'] = df.iloc[argrelextrema(
        df['Close'].values, np.less, order=order)[0]]['Close']
    df['RSI_Max'] = df.iloc[argrelextrema(
        df['RSI'].values, np.greater, order=order)[0]]['RSI']
    df['RSI_Min'] = df.iloc[argrelextrema(
        df['RSI'].values, np.less, order=order)[0]]['RSI']

    df['Bullish_Div'] = False
    df['Bearish_Div'] = False

    # Get indices of the peaks/troughs
    max_idx = argrelextrema(df['Close'].values, np.greater, order=order)[0]
    min_idx = argrelextrema(df['Close'].values, np.less, order=order)[0]

    # Detect Bearish Divergence (Price HH, RSI LH)
    for i in range(1, len(max_idx)):
        curr, prev = max_idx[i], max_idx[i-1]
        if df['Close'].iloc[curr] > df['Close'].iloc[prev] and df['RSI'].iloc[curr] < df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bearish_Div'] = True

    # Detect Bullish Divergence (Price LL, RSI HL)
    for i in range(1, len(min_idx)):
        curr, prev = min_idx[i], min_idx[i-1]
        if df['Close'].iloc[curr] < df['Close'].iloc[prev] and df['RSI'].iloc[curr] > df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bullish_Div'] = True

    return df


df = detect_signals(df)

# 3. Plotting
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05, row_heights=[0.7, 0.3])

# Subplot 1: Candlesticks and Signals
fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
              low=df['Low'], close=df['Close'], name='Price'), row=1, col=1)

# Highlight Signals on Chart
bullish_signals = df[df['Bullish_Div']]
bearish_signals = df[df['Bearish_Div']]

fig.add_trace(go.Scatter(x=bullish_signals.index, y=bullish_signals['Low'] * 0.99, mode='markers',
                         marker=dict(symbol='triangle-up', size=12, color='lime'), name='Bullish Div'), row=1, col=1)
fig.add_trace(go.Scatter(x=bearish_signals.index, y=bearish_signals['High'] * 1.01, mode='markers',
                         marker=dict(symbol='triangle-down', size=12, color='red'), name='Bearish Div'), row=1, col=1)

# Subplot 2: RSI
fig.add_trace(go.Scatter(
    x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

fig.update_layout(title=f'RSI Divergence Strategy - {symbol}',
                  template='plotly_dark', xaxis_rangeslider_visible=False, height=800)
fig.show()

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_11697/1130297225.py:11: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema


def analyze_pair(symbol="BTC-USD", interval="1h", period="60d", limit=100, order=5):
    """
    Fetches data, detects RSI divergence, and plots results dynamically.
    
    Parameters:
    - symbol: The ticker (e.g., 'AAPL', 'ETH-USD', 'EURUSD=X')
    - interval: Timeframe (1m, 5m, 1h, 1d, etc.)
    - period: Data history to fetch (1d, 5d, 1mo, 1y, etc.)
    - limit: Only show the last N candles on the chart
    - order: Sensitivity for peak detection
    """

    # 1. Setup Data
    df = yf.download(symbol, period=period, interval=interval, progress=False)
    if df.empty:
        print(f"No data found for {symbol}")
        return

    # Clean columns (handles MultiIndex columns from newer yfinance versions)
    df.columns = [col[0] if isinstance(
        col, tuple) else col for col in df.columns]

    # 2. Calculate RSI
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df.dropna(inplace=True)

    # 3. Detect Signals (Logic moved inside to maintain scope)
    # Find local peaks and troughs
    df['Price_Max'] = df.iloc[argrelextrema(
        df['Close'].values, np.greater, order=order)[0]]['Close']
    df['Price_Min'] = df.iloc[argrelextrema(
        df['Close'].values, np.less, order=order)[0]]['Close']
    df['RSI_Max'] = df.iloc[argrelextrema(
        df['RSI'].values, np.greater, order=order)[0]]['RSI']
    df['RSI_Min'] = df.iloc[argrelextrema(
        df['RSI'].values, np.less, order=order)[0]]['RSI']

    df['Bullish_Div'] = False
    df['Bearish_Div'] = False

    max_idx = argrelextrema(df['Close'].values, np.greater, order=order)[0]
    min_idx = argrelextrema(df['Close'].values, np.less, order=order)[0]

    # Bearish Divergence
    for i in range(1, len(max_idx)):
        curr, prev = max_idx[i], max_idx[i-1]
        if df['Close'].iloc[curr] > df['Close'].iloc[prev] and df['RSI'].iloc[curr] < df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bearish_Div'] = True

    # Bullish Divergence
    for i in range(1, len(min_idx)):
        curr, prev = min_idx[i], min_idx[i-1]
        if df['Close'].iloc[curr] < df['Close'].iloc[prev] and df['RSI'].iloc[curr] > df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bullish_Div'] = True

    # APPLY LIMIT: Slice the dataframe to only show the last 'limit' candles
    df = df.tail(limit)

    # 4. Plotting
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, row_heights=[0.7, 0.3])

    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                  low=df['Low'], close=df['Close'], name=f'{symbol} Price'), row=1, col=1)

    # Signals
    bullish_signals = df[df['Bullish_Div']]
    bearish_signals = df[df['Bearish_Div']]

    fig.add_trace(go.Scatter(x=bullish_signals.index, y=bullish_signals['Low'] * 0.99, mode='markers',
                             marker=dict(symbol='triangle-up', size=15, color='lime'), name='Bullish Div'), row=1, col=1)
    fig.add_trace(go.Scatter(x=bearish_signals.index, y=bearish_signals['High'] * 1.01, mode='markers',
                             marker=dict(symbol='triangle-down', size=15, color='red'), name='Bearish Div'), row=1, col=1)

    # RSI Subplot
    fig.add_trace(go.Scatter(
        x=df.index, y=df['RSI'], name='RSI', line=dict(color='orange')), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    fig.update_layout(title=f'Divergence: {symbol} ({interval}) - Last {limit} Candles',
                      template='plotly_dark', xaxis_rangeslider_visible=False, height=800)
    fig.show()


# --- CALL THE FUNCTION HERE ---
# Example 1: Bitcoin 1-hour chart, last 100 candles
analyze_pair(symbol="BTC-USD", interval="1h", period="60d", limit=100)

# Example 2: Apple Stock 15-minute chart, last 50 candles
# analyze_pair(symbol="AAPL", interval="15m", period="60d", limit=50)

# Example 3: Forex EUR/USD daily chart, last 200 candles
# analyze_pair(symbol="EURUSD=X", interval="1d", period="2y", limit=200)

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_11697/1905193758.py:23: FutureWarning:

YF.download() has changed argument auto_adjust default to True

